## Libraries

In [1]:
import pandas as pd # Data Maniputation

## Importing Data

The original dataset is stored in the 'data/external' directory within this repository. The path has been included to run the code without further modifications.

In [2]:
df = pd.read_csv('../data/external/airlines_reviews.csv')

In [3]:
df.head()

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended
0,Flight was amazing,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes
1,seats on this aircraft are dreadful,Robert Watson,2024-02-21,Singapore Airlines,True,Booking an emergency exit seat still meant h...,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no
2,Food was plentiful and tasty,S Han,2024-02-20,Singapore Airlines,True,Excellent performance on all fronts. I would...,Family Leisure,February 2024,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,yes
3,“how much food was available,D Laynes,2024-02-19,Singapore Airlines,True,Pretty comfortable flight considering I was f...,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes
4,“service was consistently good”,A Othman,2024-02-19,Singapore Airlines,True,The service was consistently good from start ...,Family Leisure,February 2024,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,yes


## Data Cleaning (excluding 'Title', 'Reviews' and 'Route' for now)
The text features 'Title', 'Reviews', and 'Route' will be preprocessed and analysed in a further stage of the project, when Sentiment Analysis with Nature Language Processing (NLP) takes place. The initial stages of the analysis will focus on the resting features.

The dataset is checked for missing values and inconsistency in data types and values. 

### Checking missing data

In [4]:
df.isnull().sum()

Title                     0
Name                      0
Review Date               0
Airline                   0
Verified                  0
Reviews                   0
Type of Traveller         0
Month Flown               0
Route                     0
Class                     0
Seat Comfort              0
Staff Service             0
Food & Beverages          0
Inflight Entertainment    0
Value For Money           0
Overall Rating            0
Recommended               0
dtype: int64

### Column Review
After an initial review, the following columns were found to contain valid and expected values and require no further processing: 
'Review Date', 'Airline', 'Type of Traveller', 'Month Flown', 'Class', 'Seat Comfort', 'Staff Service', 'Food & Beverages', 'Value For Money', 'Overall Rating'.

#### 'Verified' Column
'Verified' contains three values that are not True or False. Attending to their content, they are converted to the corresponding value. All values are then converted to 0 or 1  to improve performance and clarity of analysis.

In [5]:
df['Verified'].value_counts()

Verified
True                                                                                                                                                                                                                                           6216
False                                                                                                                                                                                                                                          1881
*Unverified*                                                                                                                                                                                                                                      1
NotVerified                                                                                                                                                                                                                                       1
we do appreciat

In [6]:
# Convert different values to 'True' or 'False'
df['Verified'] = df['Verified'].replace('NotVerified', 'False')
df['Verified'] = df['Verified'].replace('*Unverified*', 'False')
df['Verified'] = df['Verified'].replace('we do appreciate you bringing this matter to our attention. Please accept my apologies for not having met your expectations; I do hope that we can leave you and your family with a more positive impression on your future flights with us', 'True')

# Convert the values to 0 and 1
df['Verified'] = df['Verified'].replace('False', 0)
df['Verified'] = df['Verified'].replace('True', 1)

df['Verified'].value_counts()

Verified
1    6217
0    1883
Name: count, dtype: int64

#### 'Inflight Entertainment' column
All numerical rating columns range from 1 to 5, except for 'Overall Rating' going from 1 to 10. The one rating of 0 found in Inflight Entertainment is converted to 1 for consistency.

In [7]:
df['Inflight Entertainment'].value_counts()

Inflight Entertainment
5    2672
4    2284
3    1539
1     863
2     741
0       1
Name: count, dtype: int64

In [8]:
# Replacing 0 for 1
df['Inflight Entertainment'] = df['Inflight Entertainment'].replace(0,1)
df['Inflight Entertainment'].value_counts()

Inflight Entertainment
5    2672
4    2284
3    1539
1     864
2     741
Name: count, dtype: int64

#### 'Recommended' column
Recommended values of 'yes' and 'no' are converted to 0 and 1 for future analysis.

In [9]:
df['Recommended'].value_counts()

Recommended
yes    4287
no     3813
Name: count, dtype: int64

In [10]:
# Replace 'yes' and 'no' with 1 and 0 for easier analysis
df['Recommended'] = df['Recommended'].replace('yes',1)
df['Recommended'] = df['Recommended'].replace('no',0)

df['Recommended'].value_counts()

Recommended
1    4287
0    3813
Name: count, dtype: int64

## Feature Engineering
New features are created out of the existing data based on industry knowledge and experience. This new features could help identifying trends and types of customers, adding value to the analysis.

### Frequent Reviewer
Split the reviewers in four categories based on the amount of verified reviews left. The categories correspond to common airline frequent flyers programs:

- 0 (Non frequent or Blue tier) - Customer left only 1 or 0 verified reviews 
- 1 (Silver Tier)               - Between 2 and 5 verified reviews
- 2 (Gold Tier)                 - Between 6 and 15 verified reviews
- 3 (Platinum Tier)             - Over 15 verified reviews

The distribution of Frequent Reviewers obtained represents approximately the average distribution of Frequent Flyers on a normal flight.

In [11]:
# Filter verified reviews
verified_reviews = df[df['Verified'] == True]

# Count occurrences of each name in the filtered DataFrame
verified_name_counts = verified_reviews['Name'].value_counts()

# Categorization function
def categorize_verified_reviews(count):
    if count == 1:
        return 0  # Not a frequent reviewer or Blue Tier
    elif 2 <= count <= 5:
        return 1  # Silver Tier
    elif 6 <= count <= 15:
        return 2  # Gold Tier
    else:
        return 3  # Platinum Tier

# Create the new feature
df['Frequent Reviewer'] = df['Name'].map(verified_name_counts).apply(lambda x: categorize_verified_reviews(x) if pd.notnull(x) else 0)

# Drop the Name column
df = df.drop(['Name'],axis=1)

# Check distribution of the new feature
df['Frequent Reviewer'].value_counts()

Frequent Reviewer
0    6298
1    1214
2     383
3     205
Name: count, dtype: int64

### Flight Month and Flight Year
Separating 'Month Flown' into 'Flight Month' and 'Flight Year' to perform individual analysis.

Flight Month: This feature can capture seasonal trends, such as differences in travel amount or moods during holiday seasons or off-peak times.

Flight Year: This feature can help identify trends over the years and account for significant events like the COVID-19 pandemic, which had a major impact on travel patterns.

In [12]:
# Split 'Month Flown' into 'Flight Month' and 'Flight Year'
df[['Flight Month', 'Flight Year']] = df['Month Flown'].str.split(' ', expand=True)

# Convert to appropriate types
df['Flight Year'] = df['Flight Year'].astype(int)
df['Flight Month'] = pd.to_datetime(df['Flight Month'], format='%B').dt.month

# Verify correct split
df[['Month Flown', 'Flight Year','Flight Month']].head()

,Month Flown,Flight Year,Flight Month
0,December 2023,2023,12
1,February 2024,2024,2
2,February 2024,2024,2
3,February 2024,2024,2
4,February 2024,2024,2


In [13]:
# Drop the original 'Month Flown' column
df = df.drop(['Month Flown'],axis=1)

### Quick Review
Check if the review was written soon after the flight was done. It could be useful for understanding if more immediate reviews correlate with higher or lower ratings. Consider as ‘quick review’ those that were left the same month and year as the flight was done.

Limitations:
Dataset contains no information about the flight day. Considering a whole month as the time frame, reviews could potentially be innacurately classified leading to inaccuracies and biasing the analysis.

The feature could still provide some valuable insights, while keeping in mind the potential bias. 

In [14]:
# Extracting the month and year of the review
df['Review Date'] = pd.to_datetime(df['Review Date'])
df['Review Year'] = df['Review Date'].dt.year
df['Review Month'] = df['Review Date'].dt.month

# Creating the new column 'Quick Review'
df['Quick Review'] = (df['Review Year'] == df['Flight Year']) & (df['Review Month'] == df['Flight Month'])

# Dropping the temporary columns
df=df.drop(['Review Year'],axis=1)
df=df.drop(['Review Month'],axis=1)

# Converting values to 0 and 1 for consistency
df['Quick Review'] = df['Quick Review'].map({True:1,False:0})

# Checking the new columns
df['Quick Review'].value_counts()

Quick Review
1    5005
0    3095
Name: count, dtype: int64

## Reorganizing columns

In [15]:
df.columns

Index(['Title', 'Review Date', 'Airline', 'Verified', 'Reviews',
       'Type of Traveller', 'Route', 'Class', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Frequent Reviewer', 'Flight Month',
       'Flight Year', 'Quick Review'],
      dtype='object')

In [16]:
columns_reordered = ['Title', 'Reviews','Frequent Reviewer', 'Verified', 'Airline', 'Class','Type of Traveller', 
                    'Route','Review Date','Flight Year', 'Flight Month', 'Quick Review', 'Seat Comfort',
                    'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
                    'Value For Money', 'Overall Rating', 'Recommended']

df =df[columns_reordered]

## Saving clean dataset

The preprocessed dataset is saved as 'airlines_preprocessed.csv' and stored in the 'data/interim' directory within this repository. Notebooks using that dataset contain right paths to load them.

In [17]:
df.head()

,Title,Reviews,Frequent Reviewer,Verified,Airline,Class,Type of Traveller,Route,Review Date,Flight Year,Flight Month,Quick Review,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended
0,Flight was amazing,Flight was amazing. The crew onboard this fl...,0,1,Singapore Airlines,Business Class,Solo Leisure,Jakarta to Singapore,2024-03-01,2023,12,0,4,4,4,4,4,9,1
1,seats on this aircraft are dreadful,Booking an emergency exit seat still meant h...,1,1,Singapore Airlines,Economy Class,Solo Leisure,Phuket to Singapore,2024-02-21,2024,2,1,5,3,4,4,1,3,0
2,Food was plentiful and tasty,Excellent performance on all fronts. I would...,2,1,Singapore Airlines,Economy Class,Family Leisure,Siem Reap to Singapore,2024-02-20,2024,2,1,1,5,2,1,5,10,1
3,“how much food was available,Pretty comfortable flight considering I was f...,0,1,Singapore Airlines,Economy Class,Solo Leisure,Singapore to London Heathrow,2024-02-19,2024,2,1,5,5,5,5,5,10,1
4,“service was consistently good”,The service was consistently good from start ...,0,1,Singapore Airlines,Economy Class,Family Leisure,Singapore to Phnom Penh,2024-02-19,2024,2,1,5,5,5,5,5,10,1
